In [1]:
#add spex templates to selection function
#select the highest SNR spectra of each spectral type i.e 

In [2]:
import splat
import wisps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import numba
%matplotlib inline

In [3]:
splat.initializeStandards()

In [4]:
spts=np.array([splat.typeToNum(x) for x in np.arange(16, 42)])

In [5]:
#remove object from forbidden list
splat_db0=splat.searchLibrary(vlm=True, giant=False, young=False)
splat_db0['SHORTNAME']=splat_db0.DESIGNATION.apply(splat.designationToShortName)
splat_db=splat_db0[~splat_db0.SHORTNAME.isin(wisps.FORBIDDEN_LIST)].reset_index(drop=True)

In [7]:
df=splat_db[splat_db.SPEX_TYPE.isin(spts)]

In [8]:
splat_within_snr=df[df.MEDIAN_SNR.between(5, 200)]

In [10]:
import splat.database as splatdb

Could not import regions, which is required for some of the functionalities of this module.


In [11]:
#binary vetting
folder='/users/caganze/research/wisps/data/'
bindf=pd.read_csv(folder+'/binaries.txt', names=['name', ' '], sep=',')

In [12]:
groups=splat_within_snr.sort_values('MEDIAN_SNR',ascending=False).groupby('SPEX_TYPE')

In [13]:
picks=[]
for g in groups:
    picks.append(g[1].nlargest(20,'MEDIAN_SNR'  ))

In [14]:
df_picks=pd.concat(picks)

In [27]:

def create_splat_source(filename):
    try:
        #create a wisps spectrum
        #retrieve splat spectrum

        splat_s=splat.getSpectrum(filename=filename)[0]
        #spt=splat.typeToNum(splat_s.spex_type)
        splat_s.toInstrument('WFC3-G141')

        #create a source object
        s=wisps.Spectrum( 
                       wave=splat_s.wave.value, 
                       flux=splat_s.flux.value,
                      noise=splat_s.noise.value)
        
        #put it on wisps resolution
        
        s._splat_spectrum=splat_s

        s.normalize()
        return s
    except :
        return 

In [19]:
import wisps

In [20]:
df_picks['spt']=df_picks.SPEX_TYPE.apply(wisps.make_spt_number)

In [21]:
df_picks=df_picks.sort_values(by='spt')

In [22]:
#df_picks

In [28]:
%%capture
spectra=df_picks.DATA_FILE.apply(create_splat_source)

In [92]:
def plot_sp_sp(s, a, shortname):
    std=splat.STDS_DWARF_SPEX[splat.typeToNum(s.spectral_type[0])]
    std.normalize(waverange=[1.1, 1.7])
    mask2=np.logical_and(std.wave.value>0.8, std.wave.value<2.5)
    
    mask=np.logical_and(s.wave>0.8, s.wave<2.5)
    a.plot(s.wave[mask], s.flux[mask], label=shortname,linewidth=3)
    a.plot(std.wave.value[mask2], std.flux.value[mask2], linestyle='--', label='std', alpha=0.5)
    
    
    a.set_title("{} ".format(s.spectral_type ))
    a.legend()


In [33]:
(spectra.iloc[0])

anon spectrum

In [34]:
#for idx, s in enumerate(spectra.dropna().values):
#    s.normalize()
#    fig, ax=plt.subplots()
#    plot_sp_sp(s, ax, idx)
    
    
   

In [35]:
def make_wisps(splat_s):
    s=wisps.Spectrum(wave=splat_s.wave.value, 
                       flux=splat_s.flux.value,
                      noise=splat_s.noise.value)
    return s

In [40]:
ydwarfs_stds=pd.read_pickle('/users/caganze/y_standards.pkl')
ydwarfs_data=pd.read_pickle(wisps.OUTPUT_FILES+'/schneider_spectra.pkl')

In [43]:
t9s=[make_wisps(x) for x in splat.getSpectrum(spt='T9.0')]
t8s=[make_wisps(x) for x in splat.getSpectrum(spt='T8.0')]
y0=[make_wisps(x ) for x in ydwarfs_stds.spec]
y1=[make_wisps(x) for x in ydwarfs_stds.spec]
#l3s=[make_wisps(x) for x in splat.getSpectrum(spt='L3.0', binary=False, vlm=True, young=False)]


Retrieving 2 files


Retrieving 14 files



In [44]:
extras=[x for x in np.concatenate([t9s, t8s, y0, y1])]

In [45]:
retain=np.append(spectra.dropna().values, extras)
retain=np.append(retain, ydwarfs_data)

In [47]:
dfsp=pd.DataFrame()
dfsp['spectra']=retain
dfsp['spt']= [x.spectral_type for x in retain]

In [57]:
import pickle
with open(wisps.OUTPUT_FILES+'/l_t_dwarfs_spex.pkl', 'wb') as file:
    pickle.dump(retain,file)

In [58]:
rdpck=pd.read_pickle(wisps.OUTPUT_FILES+'/l_t_dwarfs_spex.pkl')

In [59]:
bindf['shortname']=bindf.name.apply(str).apply(splat.designationToShortName)

In [60]:
splat_within_snr['shortname']=splat_within_snr.DESIGNATION.apply(splat.designationToShortName)

In [61]:
bsample=splat_within_snr[ ~ (splat_within_snr.shortname.isin(bindf.shortname) |
                             np.isnan(splat_db['PARALLAX']) | (splat_within_snr['SBINARY'] =='Y'))]

In [62]:
bsample=bsample.reset_index(drop=True)

In [63]:
%%capture
bsample_spectra=bsample.DATA_FILE.apply(create_splat_source)

In [64]:
bsample['spectra']=bsample_spectra

In [65]:
from matplotlib.backends.backend_pdf import PdfPages

In [66]:
bsample=bsample.drop_duplicates(subset='shortname').reset_index(drop=True)

In [67]:
dfextr=pd.DataFrame()
#dfextr['NAME']=['J045853.90+643451.9', 'WISE1738+2732', 'WISE1738+2732']
dfextr['SPEX_TYPE']=[x.spectral_type for x in extras]
#dfextr['shortname']=['J045853.90+643451.9', 'WISE1738+2732', 'WISE1738+2732']
dfextr['spectra']=extras


In [68]:
#add T9 and y dwarfs
#dfextr
bspl=bsample.append(dfextr, ignore_index=True, verify_integrity=True, sort=True)

In [79]:
bspl=bspl[~bspl.spectra.isna()]

In [93]:
%%capture
with PdfPages(wisps.OUTPUT_FIGURES+'/multipage_binaries.pdf') as pdf:
    
    for g in np.array_split(bspl, int(len(bspl)/4)):
        fig, ax=plt.subplots(ncols=2, nrows=2)
        
        plot_sp_sp(g.spectra.iloc[0], ax[0][0], g.shortname.iloc[0])
        plot_sp_sp(g.spectra.iloc[1], ax[0][1],  g.shortname.iloc[1])
        plot_sp_sp(g.spectra.iloc[2], ax[1][0],  g.shortname.iloc[2])
        plot_sp_sp(g.spectra.iloc[3], ax[1][1],  g.shortname.iloc[3])

        pdf.savefig() 
        plt.close()

In [84]:
bspl.reset_index(drop=True).to_pickle(wisps.OUTPUT_FILES+'/binary_spex.pkl')

In [85]:
(bspl[['SOURCE_KEY', 'NAME', 'DESIGNATION', 'RA', 'DEC', 'DISCOVERY_REFERENCE',
       'OPT_TYPE', 'NIR_TYPE', 'LIT_TYPE',
       'J_2MASS',
       'H_2MASS_E', 'GRAVITY_CLASS_NIR','BINARY', 'SBINARY','SIMBAD_NAME', 'SIMBAD_SPT',
       'SIMBAD_SPT_REF', 'PARALLAX', 'PARALLAX_E', 'DISTANCE',
       'SPEX_GRAVITY_CLASSIFICATION', 
       'shortname']]).to_excel(wisps.OUTPUT_FILES+"/nonbinaries.xlsx")  